In [ ]:
import os
import json
import numpy as np
import pandas as pd
import re

import chart_studio.plotly as py
import plotly.graph_objects as go

In [ ]:
os.makedirs("figures", exist_ok=True)

In [ ]:
# equivalent = pd.read_excel("data/GHG_EmissionsData.xlsx", skiprows=4, index_col=0, sheet_name="CO2e")
carbon = pd.read_excel("data/GHG_EmissionsData.xlsx", skiprows=4, index_col=0, sheet_name="CO2")
methane = pd.read_excel("data/GHG_EmissionsData.xlsx", skiprows=4, index_col=0, sheet_name="CH4")
nitrous = pd.read_excel("data/GHG_EmissionsData.xlsx", skiprows=4, index_col=0, sheet_name="N2O")

In [ ]:
year = "2018"
for missing in ["NO", "NE", "NO,NA"]:
    carbon[year][carbon[year] == missing] = 0
    nitrous[year][nitrous[year] == missing] = 0
    methane[year][methane[year] == missing] = 0
    
    carbon.fillna(0.0, inplace=True)
    nitrous.fillna(0.0, inplace=True)
    methane.fillna(0.0, inplace=True)

In [ ]:
with open("data/structure_pretty.json", "r") as f:
    structure = json.load(f)

In [ ]:
with open("data/labels.json", "r") as f:
    label_dict = json.load(f)

In [ ]:
carbon_scaling = 1
nitrous_scaling = 298
methane_scaling = 25

In [ ]:
sources = []
targets = []
value = []
labels = []


def add_items(source, target):
    if source not in labels:
        labels.append(source)
    if target not in labels:
        labels.append(target)
    sources.append(labels.index(source))
    targets.append(labels.index(target))


for start in list(structure.keys()):
    s = structure[start]

    if type(s) is dict:
        # This is the case for Energy as it has an extra level

        for key, val in s.items():
            add_items(start, key)
            value.append(
                carbon.loc[key, year] * carbon_scaling +  nitrous.loc[key, year] * nitrous_scaling + methane.loc[key, year] * methane_scaling
            )

            if not val:
                for t in ["CO2", "N2O", "CH4"]:
                    add_items(key, t)
                    if t == "CO2":
                        value.append(carbon.loc[key, year] * carbon_scaling)
                    elif t == "N2O":
                        value.append(nitrous.loc[key, year] * nitrous_scaling)
                    elif t == "CH4":
                        value.append(methane.loc[key, year] * methane_scaling)

            for v in val:
                value.append(
                    carbon.loc[v, year] * carbon_scaling +  nitrous.loc[v, year] * nitrous_scaling + methane.loc[v, year] * methane_scaling
                )
                add_items(key, v)

                # Final level
                for t in ["CO2", "N2O", "CH4"]:
                    add_items(v, t)
                    if t == "CO2":
                        value.append(carbon.loc[v, year] * carbon_scaling)
                    elif t == "N2O":
                        value.append(nitrous.loc[v, year] * nitrous_scaling)
                    elif t == "CH4":
                        value.append(methane.loc[v, year] * methane_scaling)

    else:
        for val in s:
            add_items(start, val)
            value.append(
                abs(carbon.loc[val, year]) * carbon_scaling +  nitrous.loc[val, year] * nitrous_scaling + methane.loc[val, year] * methane_scaling
            )
            # Final level
            for t in ["CO2", "N2O", "CH4"]:
                add_items(val, t)
                if t == "CO2":
                    value.append(abs(carbon.loc[val, year]) * carbon_scaling)
                elif t == "N2O":
                    value.append(nitrous.loc[val, year] * nitrous_scaling)
                elif t == "CH4":
                    value.append(methane.loc[val, year] * methane_scaling)


In [ ]:
flows = pd.DataFrame(data={
    'source_id': sources,
    'target_id': targets,
    'source': [labels[s] for s in sources],
    'target': [labels[t] for t in targets],
    'value': value
})

In [ ]:
with open("data/link_color.json", "r") as f:
    link_colors = json.load(f)
with open("data/node_color.json", "r") as f:
    node_colors= json.load(f)
with open("data/node_position.json", "r") as f:
    node_positions= json.load(f)

In [ ]:
flows["link_color"] = [link_colors[s] for s in flows.source]

In [ ]:
fig = go.Figure(data=[go.Sankey(
    valueformat = ".1f",
    arrangement="snap",
    orientation="h",
    node=dict(
        pad=8,
        thickness=15,
        line=dict(color="black", width=0.5),
        label=[label_dict[l] for l in labels],
        color=[node_colors[l] for l in labels],
    ),
    link=dict(
        source=flows["source_id"],
        target=flows["target_id"],
        value=flows["value"],
        color=flows["link_color"],
    ))])

fig.update_layout(title_text=f"Udledning af drivhusgasser i Danmark {year}", font_size=10)
fig.show(config={"responsive": True, "displayModeBar": False})

In [ ]:
fig.write_html("figures/sankey.html", config={"responsive": True, "displayModeBar": False})

In [ ]:
fig.write_image("figures/sankey.svg", width=1920, height=1080, scale=1)
fig.write_image("figures/sankey.png", width=1920, height=1080, scale=1)